## Instructions: ##
  
1) Place this file at the root of the folder you started jupyter notebook in.  
2) Create a "data/" folder containing all turnstile data you want to parse.  

_Example Structure:_  
~~~~
/MTA_data_parser.ipynb  
/data  
/data/turnstile_180922.txt
~~~~

In [ ]:
from multiprocessing.pool import ThreadPool
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import datetime
import requests
%matplotlib inline

# Put filepath here
data_path = 'data/'
fp = data_path + 'turnstile_180922.txt'
df = pd.read_csv(fp)
df = df.rename(columns=lambda x: x.strip())

In [ ]:
# Download all log files given range

start = [2015,1,3]
end = [2018, 9, 23]
days_between = 7

def gen_dates(start, end, days_between):
    '''Use datetime module to generate a list of dates yymmdd format given a start range, end range,
    and the num of days in between each date (start range is the base)
    :start: array of [int(year), int(mon), int(day)] specifying base start date
    :end: array of [int(year), int(mon), int(day)] specifying end date of range
    :days_between: how many days between each date (starting with start date), default = 7'''
    
    start = datetime.datetime(*start)
    end = datetime.datetime(*end)
    step = datetime.timedelta(days=days_between)
    dates = []
    
    while start < end:
        dates.append(start.strftime('%y%m%d'))
        start += step
    return dates

def download_logs(date):
    '''Given date input, download log file from MTA url:
    :date: a date str yymmdd'''
    
    print('Downloading', 'turnstile_' + date + '.txt')
    url = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt'.format(date)
    response = requests.get(url)
    open('data/turnstile_' + date + '.txt', 'wb').write(response.content)

def parallel_dl(dates, threads = 8):
    '''Paralellize downloads to n threads
    :threads: number of concurrent download threads (default is 8)'''
    results = ThreadPool(threads).imap_unordered(download_logs, dates)
    
dates = gen_dates(start, end, days_between)
parallel_dl(dates)


In [ ]:
# Use to do preliminary inspection of data

def data_inspect(df, resampled=False):
    ''' Preliminary inspection of data attributes
    :resampled: If this is set to true, the data has been cleaned and some cols removed/added. Treat differently'''
    
    print("Summarize NAs:")
    print(df.isna().sum())
    
    u_station = df['STATION'].unique()
    print("\nNum Unique Stations:", len(u_station))
    print(u_station)
    
    u_desc = df['DESC'].unique()
    print("\n Unique Descriptions")
    print(u_desc)
    
    if resampled:
        print("\nUnique Time Vals per station")
        for s in u_station:
            u_time = df[df['STATION'] == s]['DATETIME'].unique()
            print('Station:', s)
            print(u_time, '\n')

    else:
        print("\nUnique Time Vals per station")
        for s in u_station:
            u_time = df[df['STATION'] == s]['TIME'].unique()
            print('Station:', s)
            print(u_time, '\n')

data_inspect(df)

In [ ]:
# Organize data before using diff() to find absolute diff in entries/exits.
# Zero out counter rollovers (discard)
# Aggregate times to ensure 4 hour interval in-between

def clean_data(df):
    '''Clean up dataframe by grouping by correct cols, finding change in counts from entries/exits,
    zeroing out implausible counts. Add a DATETIME column with a datetime object for ease of grouping
    in analysis, and to smooth irregular data reporting intervals. All of this is stored and pickled
    in a dict whose key is the unique station name, and value is a df with cols C/A, STATION, UNIT,
    SCP, DATE, TIME.
    :df: input raw df data to be cleaned'''
    
    stop_data = {}
    u_station = df['STATION'].unique()
    
    group_order = ['C/A', 'STATION', 'UNIT', 'SCP', 'DATE', 'TIME']
    # Sort by Control Area (station), Unit (Remote unit), SCP (turnstile), Date, Time
    df = df.sort_values(by=group_order)
    
    # Substract next row by prev row for magnitude of change
    df['DIFF_ENTRY'] = df['ENTRIES'].diff()
    df['DIFF_EXIT'] = df['EXITS'].diff()
    
    # Add datetime col to resample data
    df['DATETIME'] = df['DATE'] + ' ' + df['TIME']
    df['DATETIME'] = pd.to_datetime(df['DATETIME'])
    
    # Zero out turnstile errors/rollovers
    df.loc[df['DIFF_ENTRY'] < 0, 'DIFF_ENTRY'] = 0
    df.loc[df['DIFF_ENTRY'] > 4000, 'DIFF_ENTRY'] = 0
    df.loc[df['DIFF_EXIT'] > 4000, 'DIFF_EXIT'] = 0
    df.loc[df['DIFF_EXIT'] < 0, 'DIFF_EXIT'] = 0
    
    for s in u_station:
        stop_data[s] = df[df['STATION'] == s][['C/A', 'UNIT', 'SCP', 'DIFF_ENTRY', 'DIFF_EXIT', 'DATETIME']]
    
    pickle.dump(stop_data,open('stop_data', 'wb'))
    print('Done')
clean_data(df)

In [ ]:
def plot_stop_data(fn, rule="4H"):
    ''' Load and plot entry counts in pickled file containing dictionary with stop names as keys,
    and dataframe with columns C/A, UNIT, SCP, DIFF_ENTRY, DATETIME.
    :fn: this is the filename within the specified data dir above
    :rule: this is the data aggregating option, default by to sum counts by 1 Day'''
    
    data = pickle.load(open(data_path+fn, 'rb'))
    
    for stop in data:
        if stop== 'TIMES SQ-42 ST':
            stop_data = data[stop]
            stop_data = stop_data[['C/A', 'UNIT', 'SCP', 'DIFF_ENTRY', 'DATETIME']]

            # resample to ensure everything is by 4H interval (sum irregularities in recording time)
            grouped = stop_data.resample(rule, on='DATETIME', base=0).sum()
            print(stop)
            print(grouped)
            amount_entry  = list(grouped['DIFF_ENTRY'])
            plt.plot(amount_entry)
        
plot_stop_data('all_data_parsed')

In [ ]:
def first_last(df):
    # Priyanka's First-Last method
    turnstiles_df = df
    PREV_ENTRIES=turnstiles_df.groupby(['C/A','UNIT','SCP','STATION','DATE']).ENTRIES.first().reset_index()
    LAST=turnstiles_df.groupby(['C/A','UNIT','SCP','STATION','DATE']).ENTRIES.last().reset_index()
    PREV_ENTRIES['LAST']=LAST['ENTRIES']
    PREV_ENTRIES['DAILY']=PREV_ENTRIES['LAST']-PREV_ENTRIES['ENTRIES']
    ST59 = PREV_ENTRIES[PREV_ENTRIES['STATION'] == '59 ST']
    print(ST59.groupby(['DATE']).sum())
    
first_last(df)